# Neuroconnect - dMRI course

## Hands on - Preprocessing

## https://github.com/joowon2/dwi_preprocessing/blob/master/preprocessing.ipynb

## X2Go (http://wiki.x2go.org)
* To connect a graphical desktop
* Fast X11 forwarding
* Download: http://wiki.x2go.org/doku.php/download:start

* Run X2Go Client
* Create new session
<img src="figs/1_new_session_button.png" width="400">

* host `10.81.240.56`
* port `2205`
<img src="figs/2_new_session.png" width="500">

* if you already have the session, click it
<img src="figs/3_session.png" width="400">

* type your login id and password
<img src="figs/4_login.png" width="300">

* open a terminal
<img src="figs/5_terminal_icon.png" width="300">

* if you want to log out
<img src="figs/6_logout.png" width="300">

## Prepare data

* to copy the preprocessing practical data, type the following command in terminal

```
/home/NeuroConnect/Lesson_1_Preprocessing/Practical/copy_preprocessing.sh
```

## Run jupyter notebook

```
cd ~/preprocessing
jupyter notebook
```

#### and click preprocessing.ipynb file
<img src="figs/7_notebook.png" width="300">

* To exit notebook in the terminal
* Press `Ctrl + C` twice.

## jupyter notebook

* You can run commands by using notebook interface or typing them in a terminal.
* `In [ ]`: Cell block. You can excute this block by pressing Ctrl+Enter (or Shift+Enter) or clicking play button.
* `In [*]`: Cell block is running. If finished,
* `In [a integer]`


# Diffusion MRI preprocessing (using FSL)

```
cd ~/preprocessing
pwd
```

In [ ]:
cd ~/preprocessing
pwd

## dcm2niix: convert DICOM to Nifti1
* wiki: http://www.nitrc.org/plugins/mwiki/index.php/dcm2nii:MainPage
* code: https://github.com/neurolabusc/dcm2niix

#### Example: convert the DICOM files in DICOM directory to a nifti1 file using a protocal name as a filename and store in the nifti directory

```
mkdir -p nifti
dcm2niix -z y -f %p -o nifti DICOM
```


In [ ]:
mkdir -p nifti
dcm2niix -z y -f %p -o nifti DICOM

#### Check the output directory

```
ls nifti
```

In [ ]:
ls nifti

## Diffusion-weighted imaging data
* filename.nii.gz: image file
* filename.bval: b-value
* filename.bvec: gradient direction

#### pre-converted dwi files
```
ls dwi
```

In [ ]:
ls dwi

* DWI_LR_BIC_v1 : left-right DWI
* DWI_RL_BIC_v1 : right-left DWI
* DWI_LR_BIC_v1_SBRef.nii.gz : left-right single-band reference
* DWI_RL_BIC_v1_SBRef.nii.gz : right-left single-band reference

#### Open the image
```
fslview dwi/DWI_LR_BIC_v1.nii.gz dwi/DWI_RL_BIC_v1.nii.gz
```

In [ ]:
fslview dwi/DWI_LR_BIC_v1.nii.gz dwi/DWI_RL_BIC_v1.nii.gz

<img src="figs/fig_fslview.png" width="600">

## TOPUP

* Estimates non-zero off-resonance field and corrects susceptibility-induced distortions
* http://fsl.fmrib.ox.ac.uk/fsl/fslwiki/topup

#### Extract b0 image

<img src="figs/fig_topup_1.png" width="300">

* TOPUP works with an image containing b0 frames (without diffusion-weighting) of opposite phase encoding directions
* To extract b0 images, use fslroi
* `fslroi input output start_frame number_of_frames_to_extract`

```
fslroi dwi/DWI_LR_BIC_v1.nii.gz dwi/DWI_LR_b0 0 1
```


In [ ]:
fslroi dwi/DWI_LR_BIC_v1.nii.gz dwi/DWI_LR_b0 0 1

In [ ]:
fslroi dwi/DWI_RL_BIC_v1.nii.gz dwi/DWI_RL_b0 0 1

#### Using Single Band Reference (SBRef)

<img src="figs/fig_topup_2.png" width="320">

* DWI using multi-band: fast but low quality
* For TOPUP, acquire another single-band reference scan (short, 1 frame) before DWI
* compare SBRef and multi-band DWI
```
fslview dwi/DWI_LR_BIC_v1.nii.gz dwi/DWI_LR_BIC_v1_SBRef.nii.gz
fslview dwi/DWI_RL_BIC_v1.nii.gz dwi/DWI_RL_BIC_v1_SBRef.nii.gz
```

In [ ]:
fslview dwi/DWI_LR_BIC_v1.nii.gz dwi/DWI_LR_BIC_v1_SBRef.nii.gz

In [ ]:
fslview dwi/DWI_RL_BIC_v1.nii.gz dwi/DWI_RL_BIC_v1_SBRef.nii.gz

#### open both b0 images and compare them
```
fslview dwi/DWI_LR_BIC_v1_SBRef.nii.gz dwi/DWI_RL_BIC_v1_SBRef.nii.gz
```

In [ ]:
fslview dwi/DWI_LR_BIC_v1_SBRef.nii.gz dwi/DWI_RL_BIC_v1_SBRef.nii.gz

#### Merge two b0 images to make an input image for TOPUP
```
fslmerge -t dwi/DWI_lr_rl_sbref dwi/DWI_LR_BIC_v1_SBRef.nii.gz dwi/DWI_RL_BIC_v1_SBRef.nii.gz
```

In [ ]:
fslmerge -t dwi/DWI_lr_rl_sbref dwi/DWI_LR_BIC_v1_SBRef.nii.gz dwi/DWI_RL_BIC_v1_SBRef.nii.gz

#### Create acqparams.txt file
* acqparams.txt contains PE direction at the first three columns (x, y, z) and total readout time at the last column.

* total readout time: `echo_spacing_time * (#echo_spacing - 1)`

example acqparams.txt: <br>
```
-1 0 0 0.05694
1 0 0 0.05694
```
To make this acqparams file, 
```
echo -e "-1 0 0 0.05694\n1 0 0 0.05694" > topup/acqparams.txt
```

In [ ]:
echo -e "-1 0 0 0.05694\n1 0 0 0.05694" > topup/acqparams.txt

In [ ]:
cat topup/acqparams.txt

#### topup command: http://fsl.fmrib.ox.ac.uk/fsl/fslwiki/topup/TopupUsersGuide

example:  <br>
```
topup \
    --imain=dwi/DWI_lr_rl_sbref \
    --datain=topup/acqparams.txt \
    --config=b02b0.cnf \
    --out=topup/topup \
    --iout=topup/topup_warped \
    --fout=topup/topup_field \
    --interp=spline \
    --subsamp=1
```

In [ ]:
echo topup \
    --imain=dwi/DWI_lr_rl_sbref \
    --datain=topup/acqparams.txt \
    --config=b02b0.cnf \
    --out=topup/topup \
    --iout=topup/topup_warped \
    --fout=topup/topup_field \
    --interp=spline \
    --subsamp=1
echo    
ls topup_done/

#### Topup results:
* topup_done/topup_movpar.txt: movement between dwi_ap_b0 and dwi_pa_b0
* topup_done/topup_fieldcoef.nii.gz: off-resonance field
* topup_done/topup_field.nii.gz: field
* topup_done/topup_warped.nii.gz: warped (distortion-corrected) image

#### Open results
```
fslview topup_done/topup_fieldcoef.nii.gz
fslview dwi/DWI_lr_rl_sbref topup_done/topup_warped.nii.gz
```

In [ ]:
fslview topup_done/topup_fieldcoef.nii.gz

In [ ]:
fslview dwi/DWI_lr_rl_sbref topup_done/topup_warped.nii.gz

## EDDY

* corrects eddy current-induced distortion and movements
* http://fsl.fmrib.ox.ac.uk/fsl/fslwiki/eddy

#### Creating a mask

* EDDY requires a brain mask image file.
* To generate one, run bet on the topup unwarped image.
* You may use an option `-f 0.2`

```
bet topup_done/topup_warped.nii.gz topup_done/topup_warped_bet -m
```

In [ ]:
bet topup_done/topup_warped.nii.gz topup_done/topup_warped_bet -m

#### Check the BET result
```
fslview topup_done/topup_warped_bet
```

In [ ]:
fslview topup_done/topup_warped_bet

#### Merge RL and LR DWI and run EDDY
* to merge RL and LR DWI, bval, and bvec

```
fslmerge -t dwi/DWI_lr_rl dwi/DWI_LR_BIC_v1.nii.gz dwi/DWI_RL_BIC_v1.nii.gz
paste dwi/DWI_LR_BIC_v1.bval dwi/DWI_RL_BIC_v1.bval > dwi/DWI_lr_rl.bval
paste dwi/DWI_LR_BIC_v1.bvec dwi/DWI_RL_BIC_v1.bvec > dwi/DWI_lr_rl.bvec
```

In [ ]:
fslmerge -t dwi/DWI_lr_rl dwi/DWI_LR_BIC_v1.nii.gz dwi/DWI_RL_BIC_v1.nii.gz
paste dwi/DWI_LR_BIC_v1.bval dwi/DWI_RL_BIC_v1.bval > dwi/DWI_lr_rl.bval
paste dwi/DWI_LR_BIC_v1.bvec dwi/DWI_RL_BIC_v1.bvec > dwi/DWI_lr_rl.bvec

#### fslinfo
* `fslinfo` shows short infomation of a nifti file

```
fslinfo dwi/DWI_LR_BIC_v1.nii.gz
fslinfo dwi/DWI_RL_BIC_v1.nii.gz
fslinfo dwi/DWI_lr_rl.nii.gz
```


In [ ]:
fslinfo dwi/DWI_LR_BIC_v1.nii.gz

In [ ]:
fslinfo dwi/DWI_RL_BIC_v1.nii.gz

In [ ]:
fslinfo dwi/DWI_lr_rl.nii.gz

#### Creating index.txt file

<img src="figs/fig_eddy_1.png" width="320">

The row numbers in the acqparams.txt file corresponding to the input image frames. <br>
There should be the same number of numbers as the number of frames in the input image

<img src="figs/fig_eddy_2.png" width="520">

To make an index file (http://fsl.fmrib.ox.ac.uk/fsl/fslwiki/eddy/UsersGuide) <br>
```
n_lr=`fslinfo dwi/DWI_LR_BIC_v1 | grep "\<dim4" | awk '{print $2}'`
n_rl=`fslinfo dwi/DWI_RL_BIC_v1 | grep "\<dim4" | awk '{print $2}'`
indx=""
for ((i=1; i<=$n_lr; i+=1)); do indx="$indx 1"; done
for ((i=1; i<=$n_rl; i+=1)); do indx="$indx 2"; done
echo $indx > eddy/index.txt
cat eddy/index.txt
```

In [ ]:
n_lr=`fslinfo dwi/DWI_LR_BIC_v1 | grep "\<dim4" | awk '{print $2}'`
n_rl=`fslinfo dwi/DWI_RL_BIC_v1 | grep "\<dim4" | awk '{print $2}'`
indx=""
for ((i=1; i<=$n_lr; i+=1)); do indx="$indx 1"; done
for ((i=1; i<=$n_rl; i+=1)); do indx="$indx 2"; done
echo $indx > eddy/index.txt

In [ ]:
cat eddy/index.txt

#### EDDY command: http://fsl.fmrib.ox.ac.uk/fsl/fslwiki/EDDY/UsersGuide

example: <br>
```
eddy \
    --imain=dwi/DWI_lr_rl \
    --mask=topup_done/topup_warped_bet_mask \
    --bvecs=dwi/DWI_lr_rl.bvec \
    --bvals=dwi/DWI_lr_rl.bval \
    --acqp=topup_done/acqparams.txt \
    --topup=topup_done/topup \
    --index=eddy/index.txt \
    --out=eddy/eddy_unwarped
```

In [ ]:
echo eddy \
    --imain=dwi/DWI_lr_rl \
    --mask=topup_done/topup_warped_bet_mask \
    --bvecs=dwi/DWI_lr_rl.bvec \
    --bvals=dwi/DWI_lr_rl.bval \
    --acqp=topup_done/acqparams.txt \
    --topup=topup_done/topup \
    --index=eddy/index.txt \
    --out=eddy/eddy_unwarped
echo
ls eddy_done

#### Check the results
```
fslview dwi/DWI_lr_rl eddy_done/eddy_unwarped
```

In [ ]:
fslview dwi/DWI_lr_rl eddy_done/eddy_unwarped

## Gradient non-linearity distortion correction

* We cannot practice gradient unwarp, because we cannot provide the Siemens gradient coefficient file to you
* Let's compare a pre-baked image to our topup result
```
fslview topup_done/topup_warped gradient_unwarp/topup_warped_dc
```

In [ ]:
fslview topup_done/topup_warped gradient_unwarp/topup_warped_dc

# Coffee break